### Providing feedback during a run

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=os.environ.get("OPENAI_API_KEY"))
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Endless blue beneath a sky so wide,  
Whispers of waves where the mysteries hide.  
A dance of tides, in sunlight they gleam,  
The ocean sings softly, like a distant dream.  
TERMINATE
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=47), content='Endless blue beneath a sky so wide,  \nWhispers of waves where the mysteries hide.  \nA dance of tides, in sunlight they gleam,  \nThe ocean sings softly, like a distant dream.  \nTERMINATE', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='865334ab-2945-4962-9bd6-8c366ee8f6a0', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

### Providing feedback to the next run

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=os.environ.get("OPENAI_API_KEY"))
assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Write a 4-line poem about the ocean."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a 4-line poem about the ocean.
Write a 4-line poem about the ocean.
---------- assistant ----------
Waves embrace the shore with grace,  
Whispers of the deep in endless space.  
Beneath the surface, secrets gleam,  
In the ocean's heart, we find our dream.
---------- user ----------
Can you make it about a person and its relationship with the ocean
---------- assistant ----------
With every tide, I hear your call,  
In salty depths, I feel so small.  
You cradle dreams beneath your waves,  
In your vast arms, my spirit braves.
---------- user ----------

---------- assistant ----------
TERMINATE
